In [1]:
# label for gender
FEMALE = 0
MALE = 1
UNKNOWN = -1

def mapGenderToTag(gender):
    if gender == 'female':
        return FEMALE
    elif gender == 'male':
        return MALE
    else:
        return UNKNOWN

In [2]:
# stripe HTML from 知乎 answers
from bs4 import BeautifulSoup
def concatAndStriptHTMLFromContent(aaa):
    content, headline, gender = aaa
    soup = BeautifulSoup(content)
    return [headline + ' ' + soup.get_text(), mapGenderToTag(gender)]

In [8]:
# fetch data from crawled answers
import sqlite3

# prepare data
conn = sqlite3.connect('/Users/linonetwo/Desktop/repo/zhihu-nlp-playground/data/questionsAnswers.sqlite')

def answerList(gender, count, order):
    cursor = conn.execute(f"""
        SELECT answers.content, users.headline, users.gender FROM answers LEFT JOIN users
        On answers.user = users.id
        WHERE users.gender = "{gender}"
        ORDER BY answers._id {order}
        LIMIT {count}
    """)
    return list(map(concatAndStriptHTMLFromContent, cursor.fetchall()))

In [34]:
# prepare data for training
import random
import pandas as pd

size = 2000
train_data = answerList('male', size, 'DESC') + answerList('female', size, 'DESC')
eval_data = answerList('male', size, 'ASC') + answerList('female', size, 'ASC')

random.shuffle(train_data)
random.shuffle(eval_data)

# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.
train_df = pd.DataFrame(train_data, columns=['text', 'labels'])
eval_df = pd.DataFrame(eval_data)

In [35]:
train_df.head()

,text,labels
0,我来写bug了 我觉得我的要求也很高的1.首先要有上进心，要能够为了未来去打拼，而不是蹲在家...,1
1,我们陷入了一个巨大的阴谋 前面说的差不多了。总之我有一条，能当之无愧的说出“英雄所见略同。”,1
2,一个情商在线的养娃ui设计师。 嗯，我喜欢的人的身高最好170附近，最低能接受到168，最高...,0
3,我不要介绍自己 现阶段吵架了不会冷战，不会对我冷暴力有问题可以沟通我会给你理解包容尊重也没想...,0
4,死宅一枚 用睿站用哔咔用p站用逼乎用steam好了我只找得到男的了（狗头）,1


In [36]:
eval_df.head()

,0,1
0,- “have to go baby”,1
1,我是人间惆怅客，提及旧事泪纵横，落梅横笛已三更断肠声里忆平生 我坐在角落看霓虹闪烁这个城市一...,1
2,探春与黛玉的结合体 2020.1.22更 已有对象，谢谢各位兄弟们的私信。 在这里...,0
3,其实没有那么多硬性条件，看的合眼缘就行，好吧，这么说可能有些空泛。我先介绍下我自己吧，94...,0
4,房产 我也来试试缘分吧。89年，165，颜值及格。女。合肥人。离异。有一子归男方。上市房企...,0


In [37]:
from simpletransformers.classification import ClassificationModel
import sklearn


# Create a ClassificationModel
train_args={
    'sliding_window': True,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'evaluate_during_training': True,
    'no_cache': False,
    'logging_steps': 5,
    'stride': 0.8,
    'max_seq_length': 512
}
model = ClassificationModel('bert', 'bert-base-chinese', use_cuda=False, args=train_args) # You can set class weights by using the optional weight argument

# Train the model
model.train_model(train_df, eval_df=eval_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

Converting to features started. Cache is not used.
sliding_window enabled



4465 features created from 4000 samples.


Running loss: 1.348329
Converting to features started. Cache is not used.
sliding_window enabled



4000 features created from 4000 samples.



{'mcc': 0.44408935501842345, 'tp': 1513, 'tn': 1373, 'fp': 627, 'fn': 487, 'eval_loss': 0.598417389811131}

Training of bert model complete. Saved to outputs/.
Features loaded from cache at cache_dir/cached_dev_bert_512_2_4000


In [38]:
print(result)

{'mcc': 0.44408935501842345, 'tp': 1513, 'tn': 1373, 'fp': 627, 'fn': 487, 'acc': 0.7215, 'eval_loss': 0.598417389811131}


In [39]:
predictions, raw_outputs = model.predict(['上海松江，94年狮子男，老家是江苏苏中地区的，独生子，身高180cm，80kg，由于天生骨'])

print(raw_outputs)
print(predictions)

Converting to features started. Cache is not used.
sliding_window enabled



1 features created from 1 samples.



[array([[-0.53769445, -0.11950289]], dtype=float32)]
[1]
